In [5]:
from imports import *
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np

In [4]:
# from webdriver_manager.chrome import ChromeDriverManager
#
# driver = webdriver.Chrome(ChromeDriverManager().install())

ModuleNotFoundError: No module named 'webdriver_manager'

In [4]:
# # Log in to LinkedIn
# def login(driver, p)
# url = "https://www.linkedin.com/checkpoint/rm/sign-in-another-account?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin"
# # Wait support - number of seconds before timing out
# wait = WebDriverWait(driver, 10)
# driver.get(url)
# username = driver.find_element_by_id("username")
# username.send_keys("wang.meredith09@gmail.com")
# password = driver.find_element_by_id("password")
# password.send_keys(p)
# driver.find_element_by_class_name("login__form_action_container").click()

In [107]:
# Log into linkedin
def login(driver, password):
    url =  'https://www.linkedin.com/checkpoint/rm/sign-in-another-account?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin'
    wait = WebDriverWait(driver, 10)
    driver.get(url)
  
    driver.find_element("xpath",'/html/body/div/main/div[2]/div[1]/form/div[1]/input').send_keys('wang.meredith09@gmail.com')
    driver.find_element("xpath",'/html/body/div/main/div[2]/div[1]/form/div[2]/input').send_keys(password)
    driver.find_element("xpath",'/html/body/div/main/div[2]/div[1]/form/div[3]').click()

# Redirects driver to the query link
def search(driver):
    time.sleep(5)
    driver.get("https://www.linkedin.com/jobs/search/?keywords=data%20science")

# Grab the number of results that were fetched
def get_n_results(driver):
    time.sleep(10)
    results_div = driver.find_element("xpath",'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/header/div[1]/small')
    n_string = results_div.text
    n = int(n_string.split()[0].replace(',',""))
    return n 

#Finds job ul div
def get_jobs(driver):
    ul_div = driver.find_element("xpath",'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul')
    return ul_div


#Scrolls to properly load page
def scroll_down(driver):
    for i in np.linspace(0,1,10):
        time.sleep(2)
        # Excute JavaScript
        # driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight*{i})")

def get_job_urls(jobs,driver,job_urls = {}):
    i = 1
    while True: 
        try:

            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH,f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]')))
            url = jobs.find_element("xpath",f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[1]/a').get_attribute("href")
            role = jobs.find_element("xpath",f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[1]/a').text
            company = jobs.find_element("xpath",f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[2]/a').text
            location = driver.find_element("xpath",f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[3]/ul/li').text
            job_urls.update({url:{'company':company,'location':location,'role':role}})
            i+=1
        except:
            return job_urls

def load_next_page(driver):
    curr= driver.find_element("xpath",'//*[@aria-current="true"]').text
    next = driver.find_element("xpath",f'//*[@aria-label="Page {int(curr)+1}"]')
    next.click()

def get_description(driver,job_dict):
    good = []
    fail = []
  #Iterate through the url list to scrape the descriptions
    for url in list(job_dict.keys()):
        if url not in good:
            try:
                driver.get(url)
                time.sleep(2)
                if driver.current_url != url:
                    print(f'failed at {url}')
              #remove broken urls
                    job_dict.pop(url)
              #scrape
                # driver.find_element("xpath",'//*[@aria-label="Click to see more description"]').click()
                description = driver.find_element("xpath",'/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[2]/article/div/div[1]').text
                time.sleep(3)
                job_dict.get(url).update({"description":description})
                time.sleep(6)
                good.append(url)
                return job_dict
            except:
                #keep going if there is a random error in which a div did not load properly but check where we failed
                print(f"fail {job_dict.get(url)}")
                fail.append(url)

def get_skills(driver,job_dict):
    good = []
    fail = []
    #Iterate through the url list to scrape the descriptions
    for url in list(job_dict.keys()):
        if url not in good:
            try:
                driver.get(url)
                time.sleep(2)
                if driver.current_url != url:
                    print(f'failed at {url}')
                    #remove broken urls
                    job_dict.pop(url)
                #scrape
                skill_list = driver.find_element("xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[6]/div/div[1]/div[2]/div[2]/ul')
                skills = skill_list.find_elements(By.TAG_NAME, "li")
                # if len(skills) > 0:
                # i = 1
                skill = []
                # while True:
                #     try:
                #         # WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH,f'/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[6]/div/div[1]/div[2]/div[2]/ul/li[{i}]')))
                #         skillname = driver.find_element("xpath",f'/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[6]/div/div[1]/div[2]/div[2]/ul/li[{i}]/p').text
                #         job_dict.get(url).update({"skills":skillname})
                #         i+=1
                #         good.append(url)
                #     except:
                #         return job_dict
                for i in skills:
                    skill = i.find_elements("xpath", f'//*[@id="ember95"]/div/div[1]/div[2]/div[2]/ul/li[{i}]/p').text
                    job_dict.get(url).update({"skills":skill})
                    good.append(url)
                return job_dict
            except:
                #keep going if there is a random error in which a div did not load properly but check where we failed
                print(f"fail{job_dict.get(url)}")
                fail.append(url)

In [108]:
def main(driver,password):
    login(driver, password)
    search(driver)
    n = get_n_results(driver)
    job_dict = {}
    #iterate through the amount of pages given
    for i in range(2):
        jobs = get_jobs(driver)
        scroll_down(driver)
        get_job_urls(jobs,driver,job_urls = job_dict)
        time.sleep(1)
        load_next_page(driver)
    get_skills(driver,job_dict)
    df = pd.DataFrame(job_dict)
    # return get_description(driver,job_dict,good)
    return df

In [109]:
secret_file = open("env.txt", "r")
password = secret_file.read()
type(password)

str

In [110]:
driver = webdriver.Chrome('/Users/williambaldridge/linkedin-data-science-project/chromedriver')
secret_file = open("env.txt", "r")
password = secret_file.read()
main(driver,password)

# driver.quit()

fail{'company': 'TELUS International', 'location': 'Colorado, United States', 'role': 'Digital Solutions | Data Science Engineer with ML'}
fail{'company': 'PwC', 'location': 'Austin, TX', 'role': 'Data Science & Machine Learning SA w/ Conversational AI'}
fail{'company': 'Johnson & Johnson', 'location': 'Cincinnati, OH', 'role': 'Assistant/Associate Engineer, 2023 Engineering Development Program (EDP), Data Science Engineer'}
fail{'company': 'Booz Allen Hamilton', 'location': 'Huntsville, AL', 'role': 'Data Science Developer'}
fail{'company': 'Expedia Group', 'location': 'Austin, TX', 'role': 'Data Scientist II, Product Analytics'}
fail{'company': 'Elastic', 'location': 'San Antonio, TX', 'role': 'Analytics Engineer'}
fail{'company': 'Codeup', 'location': 'San Antonio, Texas Metropolitan Area', 'role': 'Data Science Instructor'}


,https://www.linkedin.com/jobs/view/3155711956/?eBP=CwEAAAGCn9jCNoky5gZiIoJzY7Tal2Y3YBYPJIvepgDfeQl8IjJ2tOUSkrokVwv0ZHK19cfmg2sSrxyPj-4qWJGVn1s4qJNQOwEqgp7crkzqT4Qr_aLQ-UCGuyz2OWhRlycLEkxMWOiZWnPsHkU4xPtCWmlxtlFEG8tXPuofb5hSjBSPj0B4QvkYUTunK6mUxMhHotcQJ5o7xmkTsC3EulovffHXcFEGJzrvCss-8iZWdKD9KZDmJicSCoAamLHsxkCzQAV6kaeN2xtlEfAT3y47FsI9a0NC-r6l7e9NHNWdCO8YPpcDytdk9ZK9rp-jVOfkOU7tUBDdcESqAcyZj9Azu5583U9P1wIH2r7fRPzBcKQAC2cHSdaSTDlCgGrz&recommendedFlavor=SKILL_ASSESSMENTS&refId=jM228hEgtvGOYmgO16rhnQ%3D%3D&trackingId=cyJYuq69Xr5mXs%2BSRaSB9w%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3142891690/?eBP=CwEAAAGCn9jCNpWxw8E_OqE-fvjMLg-WoethgIG-Kr1u7k8IWQy1qfc3mOnE-6MFrmn1N1B2I3ydEsJcR25k7OlnlEIvMVCzyqvuh__J1CjXyPZS07ePrzhBPlLeX7WZXv1eBQxNRBBfPSUiJH3teW-K_qg5mKDPUBb6F8tmEsISC4qry9mDTAtqK4TQW3H19d7vtrAwnJBH0pfddZyjuvNnZfzI2_AIg7hUMR4B_ZE2ouxg-x0vqBnrlny1BtzArtz6LI_FP-dLjEDYc2v9seia-25MjB-nXfjrSTErLWd5YllpOn_ZAjk91vhsaLAGzURmZFb99niWx7o0Y7zzi9ffU91c1Twb-2qzMmHdbf7KyL0R29r113V1Yi3Zoa8yggA&recommendedFlavor=SKILL_ASSESSMENTS&refId=jM228hEgtvGOYmgO16rhnQ%3D%3D&trackingId=GH62gzJlUZbSS5B5qdOSbw%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3192667302/?eBP=CwEAAAGCn9jCNjRddQOoMQTL1XTp33VedODewuO7NV8Hb3L-byz7fNrR2dg2ySE-Bdu6cet-uUg9GyjU0uAgcjEd7LreDvbv7C6Ph1YGeuLBVWFxR95NL0yVRN-aW0mcgO6xKcpYuLbFWm0icyzTyjttF2542_VEgqngdSVsbyAos7j9-7QlyDxNIx4JjyRTs6Xm0AdQ_jjn8A1tao1fDqtUptLSKGaojKY4udor0i8rfV7YXiZk80oQIv8fCvE0dhOJkCWXousWzPEqGqK7TLae0klVb9dPyBOOHRBXsFmmls7P2mAhGuUxNVnxwhgXRaHgSyKvUs0eetO1c5Dh2V1I97UVR9hsbBMfqK0RHjATAHHE8GSeSnpDSGeo50ld&recommendedFlavor=SKILL_ASSESSMENTS&refId=jM228hEgtvGOYmgO16rhnQ%3D%3D&trackingId=p2GXErocXYSXcTzFuDyfMQ%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3217341688/?eBP=CwEAAAGCn9jCNmgBaZfbTNygHZKyLnUju52toLIRKqshry7KdA6w7aLbpfNjIU1sjl2JX-aSDYw-FIAHGDaC2QzIbaNg4wqrOODexmINomyuZeBCYpTpxpdbEQ9BnUZlr6iypcxjAQsHNLis0YQsboxKYk2EzUCa2AFkThFhPn9GyqT9_2MtsIBIqU3UPVhtsONCzGZuMiCdXwprE19FG_OehwR5rCaq4PH8bfnVUy8HA6xYF5gbD6hnzbgi0jfgQQqjjHhmHrUBtKPNX0voydYkTyn9C4EKIEJfFoTGBBQa86wrkhSxgA5IrgkZnLUWyCzGIjZJbNpEDRN3asq0euPQe3bXg9fGiJG3Jl8q9Dlr2enWO9V0AwgQ2X-xbMUX4H4&recommendedFlavor=SKILL_ASSESSMENTS&refId=jM228hEgtvGOYmgO16rhnQ%3D%3D&trackingId=5PhDtANinEm%2F%2F%2BORZr1A4A%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3213166440/?eBP=JOB_SEARCH_ORGANIC&recommendedFlavor=IN_NETWORK&refId=jM228hEgtvGOYmgO16rhnQ%3D%3D&trackingId=QsGOeYi7xb%2B70OFgSdNggg%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3214120458/?eBP=JOB_SEARCH_ORGANIC&recommendedFlavor=SKILL_ASSESSMENTS&refId=jM228hEgtvGOYmgO16rhnQ%3D%3D&trackingId=B1Ii4J6JvLzVwRW2s1xNsw%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3184747831/?eBP=CwEAAAGCn9jCNlMPubjmW2Z4X4RzkpR97TjWMXl-vjgNnYBkVNhxtg6XtEM0_OgM-xndA_uOUfCiP1925s18JBip2cdMPpapxt-sMG3UG72nnJo6UpgwQnYxT2_hkGsBv1VMUYt-c3RJBEVqAHbWfGMx584b231snV1fFNvK_O1qkSqULg7vsHABh664zTBCs13KoIEMQjQEPX5YvUg18tCeNKcy8rz9MZW-Byqk_sETPYQzvnPcWFH9Us1A6MKsfn4t-hUqapdTaQt1XczgZgFTdIyLtFQ73udEpLce-WR7jkArd1nNUp_s9FHODCwMzTDuYuGd6CoDev8lSueveuAx35SIYEfzvY9-Gb3LFiYewBuFpjGkUHFDjvlm248J&recommendedFlavor=SCHOOL_RECRUIT&refId=jM228hEgtvGOYmgO16rhnQ%3D%3D&trackingId=zJBCfN1HYzq6owyR5aAkrQ%3D%3D&trk=flagship3_search_srp_jobs
company,TELUS International,PwC,Johnson & Johnson,Booz Allen Hamilton,Expedia Group,Elastic,Codeup
location,"Colorado, United States","Austin, TX","Cincinnati, OH","Huntsville, AL","Austin, TX","San Antonio, TX","San Antonio, Texas Metropolitan Area"
role,Digital Solutions | Data Science Engineer with ML,Data Science & Machine Learning SA w/ Conversational AI,"Assistant/Associate Engineer, 2023 Engineering Development Program (EDP), Data Science...",Data Science Developer,"Data Scientist II, Product Analytics",Analytics Engineer,Data Science Instructor
